In [6]:
import yfinance as yf
import pandas as pd
from IPython.display import display
from scipy.stats import zscore
import numpy as np
import torch
import gym
from gym import spaces
from envs.portfolio_env import Mult_Asset_portEnv
import random

from stable_baselines3 import A2C  
from stable_baselines3.common.vec_env import DummyVecEnv

from stable_baselines3 import PPO
 

In [2]:
assets = ['NVDA', 'AAPL', 'AMZN', 'JPM', 'IBM', 'MSFT', 'TSLA', 'GOOGL', 'META', 'HSBC']
start_date ='2019-01-01'
end_date = '2025-01-01'
 
df = yf.download(tickers=assets, start= start_date , end= end_date, interval='1d')

df 

/tmp/ipykernel_6066/1496721472.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=assets, start= start_date , end= end_date, interval='1d')
[*********************100%***********************]  10 of 10 completed


Price            Close                                                 \
Ticker            AAPL        AMZN       GOOGL       HSBC         IBM   
Date                                                                    
2019-01-02   37.617855   76.956497   52.419624  27.954201   82.193146   
2019-01-03   33.870834   75.014000   50.967831  27.646481   80.552284   
2019-01-04   35.316746   78.769501   53.582150  28.453386   83.698479   
2019-01-07   35.238152   81.475502   53.475292  28.070448   84.290596   
2019-01-08   35.909893   82.829002   53.944973  28.145664   85.489143   
...                ...         ...         ...        ...         ...   
2024-12-24  257.578674  229.050003  195.647552  47.340084  221.451202   
2024-12-26  258.396667  227.050003  195.138748  47.349697  221.924866   
2024-12-27  254.974930  223.750000  192.305435  47.378536  219.842682   
2024-12-30  251.593079  221.300003  190.789047  47.590000  217.346039   
2024-12-31  249.817368  219.389999  188.853607  47.541943  216.931580   

Price                                                                   ...  \
Ticker             JPM        META        MSFT        NVDA        TSLA  ...   
Date                                                                    ...   
2019-01-02   82.408615  134.939713   94.945518    3.377544   20.674667  ...   
2019-01-03   81.237457  131.021194   91.452667    3.173484   20.024000  ...   
2019-01-04   84.232300  137.197311   95.706055    3.376801   21.179333  ...   
2019-01-07   84.290863  137.296783   95.828125    3.555572   22.330667  ...   
2019-01-08   84.131905  141.752350   96.522911    3.467054   22.356667  ...   
...                ...         ...         ...         ...         ...  ...   
2024-12-24  239.589218  606.742920  437.647400  140.197372  462.279999  ...   
2024-12-26  240.409912  602.350220  436.432068  139.907410  454.130005  ...   
2024-12-27  238.462036  598.816040  428.881104  136.987885  431.660004  ...   
2024-12-30  236.632812  590.260254  423.202911  137.467804  417.410004  ...   
2024-12-31  237.018433  584.539795  419.885681  134.268326  403.839996  ...   

Price          Volume                                                   \
Ticker           AAPL       AMZN     GOOGL     HSBC      IBM       JPM   
Date                                                                     
2019-01-02  148158800  159662000  31868000  2565000  4434935  15670900   
2019-01-03  365248800  139512000  41960000  1479400  4546648  16286400   
2019-01-04  234428400  183652000  46022000  3060900  4683779  16935200   
2019-01-07  219111200  159864000  47446000  2212300  3923755  15430700   
2019-01-08  164101200  177628000  35414000  2888300  4982726  13578800   
...               ...        ...       ...      ...      ...       ...   
2024-12-24   23234700   15007500  10403300   309000  1186200   3729100   
2024-12-26   27237100   16146700  12046600   516300  3286500   4451800   
2024-12-27   42355300   27367100  18891400  1088100  1810800   5730200   
2024-12-30   35557500   28321200  14264700  1344500  2095600   5723800   
2024-12-31   39480700   24819700  17466900   768200  2270200   4871000   

Price                                                 
Ticker          META      MSFT       NVDA       TSLA  
Date                                                  
2019-01-02  28146200  35329300  508752000  174879000  
2019-01-03  22717900  42579100  705552000  104478000  
2019-01-04  29002100  44060600  585620000  110911500  
2019-01-07  20089300  35656100  709160000  113268000  
2019-01-08  26263800  31514400  786016000  105127500  
...              ...       ...        ...        ...  
2024-12-24   4726100   7164500  105157000   59551800  
2024-12-26   6081400   8194200  116205600   76366400  
2024-12-27   8084200  18117700  170582600   82666800  
2024-12-30   7025900  13158700  167734700   64941000  
2024-12-31   6019500  13246500  155659200   76825100  

[1510 rows x 50 columns]

In [3]:
sta = df.stack()

df_flt = sta.reset_index()
is_multi_index = isinstance(df_flt.index, pd.MultiIndex)  
is_multi_col= isinstance(df_flt.columns, pd.MultiIndex)

print (df_flt.head())
print (df_flt.info())
print (df_flt.describe())
print (df_flt.dtypes)
print (df_flt.isna().sum())
print(df_flt.index.to_series().min())
print(df_flt.index.to_series().max())
print ('Row', is_multi_index)
print ('col', is_multi_col)

/tmp/ipykernel_6066/3544417225.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  sta = df.stack()


Price       Date Ticker      Close       High        Low       Open     Volume
0     2019-01-02   AAPL  37.617855  37.839391  36.738866  36.896084  148158800
1     2019-01-02   AMZN  76.956497  77.667999  73.046501  73.260002  159662000
2     2019-01-02  GOOGL  52.419624  52.723303  50.958387  51.053815   31868000
3     2019-01-02   HSBC  27.954201  27.974714  27.523399  27.605456    2565000
4     2019-01-02    IBM  82.193146  82.742479  79.681910  79.910201    4434935
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15100 entries, 0 to 15099
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    15100 non-null  datetime64[ns]
 1   Ticker  15100 non-null  object        
 2   Close   15100 non-null  float64       
 3   High    15100 non-null  float64       
 4   Low     15100 non-null  float64       
 5   Open    15100 non-null  float64       
 6   Volume  15100 non-null  int64         
dtypes: datetime64[ns]

In [4]:
# for symbol in assets:
#     df = yf.download(tickers=symbol, start= start_date , end= end_date, interval='1d')
#     sta = df.stack()
#     df_flt = sta.reset_index()
#     df_flt.to_csv(f'/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/raw/{symbol}_daily.CSV')


In [5]:

appl = pd.read_csv('/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/raw/AAPL_daily.CSV', parse_dates=["Date"], index_col="Date")
msft = pd.read_csv('/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/raw/MSFT_daily.CSV', parse_dates=["Date"], index_col="Date")


df = pd.concat([appl, msft])


is_multi_index = isinstance(df.index, pd.MultiIndex)  
is_multi_col= isinstance(df.columns, pd.MultiIndex)

df = df.drop(columns=['Unnamed: 0']).copy()

print (df.head())
print (df.columns)
print (df[['Ticker']])
print (df.info())
print (df.describe())
print (df.dtypes)
print (df.isna().sum())
print (df.index.to_series().min())
print (df.index.to_series().max())
print ('Row', is_multi_index)
print ('col', is_multi_col)

           Ticker      Close       High        Low       Open     Volume
Date                                                                    
2019-01-02   AAPL  37.617844  37.839379  36.738855  36.896073  148158800
2019-01-03   AAPL  33.870834  34.711709  33.825574  34.297226  365248800
2019-01-04   AAPL  35.316757  35.385840  34.254350  34.428241  234428400
2019-01-07   AAPL  35.238152  35.452541  34.754589  35.421573  219111200
2019-01-08   AAPL  35.909897  36.164781  35.378693  35.626428  164101200
Index(['Ticker', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
           Ticker
Date             
2019-01-02   AAPL
2019-01-03   AAPL
2019-01-04   AAPL
2019-01-07   AAPL
2019-01-08   AAPL
...           ...
2024-12-24   MSFT
2024-12-26   MSFT
2024-12-27   MSFT
2024-12-30   MSFT
2024-12-31   MSFT

[3020 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3020 entries, 2019-01-02 to 2024-12-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  D

In [6]:
df['LogReturn'] = df.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(1)))

df.dropna(subset=['LogReturn'], inplace=True)

df['LogReturn_Z'] = df.groupby('Ticker')['LogReturn'].transform(lambda x: (x - x.mean())/ x.std())

print (df.head())
print (df.isna().sum())
print (df[df.isna().any(axis=1)])
print (df.tail())


           Ticker      Close       High        Low       Open     Volume  \
Date                                                                       
2019-01-03   AAPL  33.870834  34.711709  33.825574  34.297226  365248800   
2019-01-04   AAPL  35.316757  35.385840  34.254350  34.428241  234428400   
2019-01-07   AAPL  35.238152  35.452541  34.754589  35.421573  219111200   
2019-01-08   AAPL  35.909897  36.164781  35.378693  35.626428  164101200   
2019-01-09   AAPL  36.519714  36.810329  35.643110  36.038533  180396400   

            LogReturn  LogReturn_Z  
Date                                
2019-01-03  -0.104924    -5.465055  
2019-01-04   0.041803     2.087049  
2019-01-07  -0.002228    -0.179263  
2019-01-08   0.018884     0.907367  
2019-01-09   0.016839     0.802147  
Ticker         0
Close          0
High           0
Low            0
Open           0
Volume         0
LogReturn      0
LogReturn_Z    0
dtype: int64
Empty DataFrame
Columns: [Ticker, Close, High, Low, Open, V

In [7]:

# df.to_csv('/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_daily.CSV')


In [8]:
addres = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_daily.CSV'
raw = pd.read_csv(addres, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')

def shrinking_ints (df):
    mapping = {}
    for col in df.dtypes [df.dtypes=='int64[pyarrow]'].index:
        max_ = df[col].max()
        min_ = df[col].min()
        if min_ < 0:
            continue
        elif max_ < 255:
            mapping[col] = 'uint8[pyarrow]'
        elif max_ <65_535:
            mapping[col] = 'uint16[pyarrow]'
        elif max_ < 4294967295:
            mapping[col] = 'uint32[pyarrow]'
    return df.astype(mapping)

def clean(df):
    return (df
    .assign(**df.select_dtypes('string').replace('', 'Missing').astype('category'))
    .pipe(shrinking_ints)
    )
    
df = clean(raw)



In [9]:
feat_to_pivot = ['Close', 'High', 'Low', 'Open', 'Volume', 'LogReturn', 'LogReturn_Z']

pivoted_featu = []

for feature in feat_to_pivot:
    feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
    feat_pivot.columns = [f"{ticker}_{feature}" for ticker in feat_pivot.columns]
    pivoted_featu.append(feat_pivot)
    
df_wide = pd.concat(pivoted_featu, axis=1)

org_rows = len(df_wide)
df_wide.dropna(inplace=True)
rows_after_drop = len(df_wide)
if org_rows != rows_after_drop:
    print(1)
    
df_wide = df_wide.sort_index(axis=1)

print (df_wide.head())
print (df_wide.info())
print (df_wide.columns)

/tmp/ipykernel_6066/607006344.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
/tmp/ipykernel_6066/607006344.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
/tmp/ipykernel_6066/607006344.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
/tmp/ipykernel_6066/60700634

            AAPL_Close  AAPL_High  AAPL_LogReturn  AAPL_LogReturn_Z  \
Date                                                                  
2019-01-03   33.870834  34.711709       -0.104924         -5.465055   
2019-01-04   35.316757   35.38584        0.041803          2.087049   
2019-01-07   35.238152  35.452541       -0.002228         -0.179263   
2019-01-08   35.909897  36.164781        0.018884          0.907367   
2019-01-09   36.519714  36.810329        0.016839          0.802147   

             AAPL_Low  AAPL_Open  AAPL_Volume  MSFT_Close  MSFT_High  \
Date                                                                   
2019-01-03  33.825574  34.297226  365248800.0   91.452667  94.072308   
2019-01-04   34.25435  34.428241  234428400.0   95.706047  96.250633   
2019-01-07  34.754589  35.421573  219111200.0    95.82811  96.964225   
2019-01-08  35.378693  35.626428  164101200.0   96.522919  97.621475   
2019-01-09   35.64311  36.038533  180396400.0   97.903183  98.475936  

In [10]:
# output = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_daily.CSV"
# df_wide.to_csv(output)

In [2]:
addres = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_daily.CSV'
df = pd.read_csv(addres, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')


try:
    df = pd.read_csv(addres, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')

    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].astype(np.float32)


except FileNotFoundError:
    print(f"Error: The wide-format data file '{addres}' was not found.")
    print("Please ensure you have run the data processing steps to create this file correctly.")
    exit() 

features_list = ['Close', 'High', 'Low', 'Open', 'Volume', 'LogReturn', 'LogReturn_Z']

# Dynamically determine the number of assets from the columns of df_wide
all_tickers = sorted(list(set([col.split('_')[0] for col in df.columns if '_' in col])))
num_assets = len(all_tickers)

# --- Environment Initialization ---
print("--- Initializing Multi-Asset Portfolio Environment for A2C Training ---")


--- Initializing Multi-Asset Portfolio Environment for A2C Training ---


In [12]:
env = Mult_Asset_portEnv(df=df,
                         num_assets=num_assets,
                         features_list=features_list,
                         window_size=5)

# 5. Run the simulation
state = env.reset()

print("\n--- Starting Multi-Asset Environment Test ---")

for _ in range(10):
    action = env.action_space.sample()
    action = action / np.sum(action)
    next_step, reward, done, info = env.step(action)
    env.render()
    
    if done:
        print('Episode finished.')
        break


--- Starting Multi-Asset Environment Test ---
Step 6, Total Portfolio Value: 999.35, Cash: 349.57, Weights: ['0.38', '0.27', '0.35']
Step 7, Total Portfolio Value: 992.78, Cash: 263.87, Weights: ['0.05', '0.69', '0.27']
Step 8, Total Portfolio Value: 986.40, Cash: 403.48, Weights: ['0.34', '0.25', '0.41']
Step 9, Total Portfolio Value: 1000.06, Cash: 262.67, Weights: ['0.20', '0.54', '0.26']
Step 10, Total Portfolio Value: 1004.04, Cash: 614.15, Weights: ['0.10', '0.29', '0.61']
Step 11, Total Portfolio Value: 1006.33, Cash: 617.46, Weights: ['0.26', '0.12', '0.61']
Step 12, Total Portfolio Value: 1009.42, Cash: 219.13, Weights: ['0.61', '0.17', '0.22']
Step 13, Total Portfolio Value: 992.01, Cash: 46.80, Weights: ['0.84', '0.11', '0.05']
Step 14, Total Portfolio Value: 995.83, Cash: 429.69, Weights: ['0.34', '0.23', '0.43']
Step 15, Total Portfolio Value: 991.88, Cash: 306.33, Weights: ['0.49', '0.20', '0.31']


In [ ]:

env = DummyVecEnv([lambda: Mult_Asset_portEnv(df=df,
                                             num_assets=num_assets,
                                             features_list=features_list,
                                             window_size=5)])

# --- A2C Model Initialization and Training ---
print("\n--- Initializing A2C Model and Starting Training ---")
model_A2C = A2C('MlpPolicy', env, verbose=1)

model_A2C.learn(total_timesteps=10000)

print("\n--- A2C Model Training Finished ---")
print("You can now evaluate the trained model_A2C or save it for later use.")

DEBUG: Observation dtype at step 5: float32

--- Initializing A2C Model and Starting Training ---
Using cpu device


/home/micheal/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step

In [7]:
print ('\n--- Initializing PPO Model and Starting Training')

model_ppo = PPO('MlpPolicy', env, verbose=1)

model_ppo.learn(total_timesteps=1000)

print("\n--- PPO Model Training Finished ---")
print("You can now evaluate the trained PPO model or save it for later use.")


--- Initializing PPO Model and Starting Training
Using cpu device
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Observation dtype at step 6: float32
DEBUG: Observation dtype at step 7: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 8: float32
DEBUG: Observation dtype at step 5: float32
DEBUG: Ob